In [19]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import json
from collections import defaultdict

Example of loading details from steamspy and steamcharts by id:

In [37]:
id = 1086940 #bg3  
details = json.loads(requests.get(f"https://steamspy.com/api.php?request=appdetails&appid={id}").text)
print("details:", details)
timestamps = json.loads(requests.get(f"https://steamcharts.com/app/{id}/chart-data.json").text)
print(f"{len(timestamps)} timestamps total, from {datetime.fromtimestamp(timestamps[0][0]/1000)}, to {datetime.fromtimestamp(timestamps[-1][0]/1000)}")

details: {'appid': 21120, 'name': 'F.E.A.R.', 'developer': 'Monolith Productions, Inc., Timegate', 'publisher': 'Warner Bros. Games, Warner Bros. Interactive Entertainment', 'score_rank': '', 'positive': 0, 'negative': 0, 'userscore': 0, 'owners': '1,000,000 .. 2,000,000', 'average_forever': 495, 'average_2weeks': 0, 'median_forever': 599, 'median_2weeks': 0, 'price': '5499', 'initialprice': '5499', 'discount': '0', 'ccu': 17, 'languages': 'English', 'genre': 'Action', 'tags': []}
885 timestamps total, from 2012-07-01 03:00:00, to 2023-12-03 23:08:33


Crawl csv's from steamspy and steamcharts:

In [44]:
def load(page=0, max=1_000_000, overwrite = True):
    if overwrite:
        details_df = pd.DataFrame(columns=["name", "developer", "publisher", "positive", "negative", "min_owners", "max_owners", "average_forever", "average_2weeks",
                          "median_forever", "median_2weeks", "ccu", "price", "initialprice", "discount", "genre", "tags", "languages"])  # https://steamspy.com/api.php
        timestamps_df = pd.DataFrame(columns=pd.date_range(start=datetime(2012, 7, 1), end=datetime.now(), freq='MS'), dtype=int)  # https://steamcharts.com/about
    else:
        details_df = pd.read_csv("details.csv", index_col=0)
        timestamps_df = pd.read_csv("timestamps.csv", index_col=0)
        return details_df, timestamps_df

    while (True):
        details_df.to_csv("details.csv")
        timestamps_df.to_csv("timestamps.csv")
        print("Starting pargsing page:", page)
        request = requests.get(f"https://steamspy.com/api.php?request=all&page={page}")
        if request.status_code != 200:
            print("Exiting on page:", page)
            return details_df, timestamps_df
        for id, _ in json.loads(request.text).items():
            if max <= 0:
                details_df.to_csv("details.csv")
                timestamps_df.to_csv("timestamps.csv")
                print("max allowed iterations is exceeded")
                return details_df, timestamps_df
            try:
                details = json.loads(requests.get(f"https://steamspy.com/api.php?request=appdetails&appid={id}").text)
                owners = details['owners'].replace(',', '').split(' .. ')
                details['min_owners'] = int(owners[0])
                details['max_owners'] = int(owners[1])
                details['genre'] = details['genre'].split(', ')
                details['languages'] = details['languages'].split(', ')
                details['tags'] = list(details['tags'].keys()) if len(details['tags']) > 0 else []
                details['price'] = int(details['price'])
                details['initialprice'] = int(details['initialprice'])
                details['discount'] = int(details['discount'])
                details_df.loc[id] = details
            except Exception as e:
                print(f"(steamspy), page: {page}, id: {id}", e)

            try:
                timestamps = json.loads(requests.get(f"https://steamcharts.com/app/{id}/chart-data.json").text)
                monthly_avgs = defaultdict(list)
                for pair in timestamps:
                    date = datetime.fromtimestamp(pair[0] / 1000)
                    monthly_avgs[datetime(date.year, date.month, 1)].append(pair[1])
                timestamps_df.loc[id] = {m: float(np.max(v)) for m, v in monthly_avgs.items()}
            except Exception as e:
                print(f"(steamcharts), page: {page}, id: {id}", e)

            max -= 1
        page += 1
        

details_df, timestamps_df = load()



print(details_df.dtypes)
print(timestamps_df.dtypes)
print("---------------------------------------------------------------")
print(details_df)
print("---------------------------------------------------------------")
print(timestamps_df)

name               object
developer          object
publisher          object
positive            int64
negative            int64
min_owners          int64
max_owners          int64
average_forever     int64
average_2weeks      int64
median_forever      int64
median_2weeks       int64
ccu                 int64
price               int64
initialprice        int64
discount            int64
genre              object
tags               object
languages          object
dtype: object
2012-07-01    float64
2012-08-01    float64
2012-09-01    float64
2012-10-01    float64
2012-11-01    float64
               ...   
2023-08-01    float64
2023-09-01    float64
2023-10-01    float64
2023-11-01    float64
2023-12-01    float64
Length: 138, dtype: object
---------------------------------------------------------------
                                     name              developer  \
570                                Dota 2                  Valve   
730      Counter-Strike: Global Offensive        

In [41]:
# from steam import Steam
# from os import environ
# from dotenv import load_dotenv

# load_dotenv()
# KEY = environ["STEAM_API_KEY"]
# steam = Steam(KEY)

# print(steam.apps.get_app_details(70))
# print(steam.apps.search_games("baldurs gate 3"))

{'70': {'success': True, 'data': {'type': 'game', 'name': 'Half-Life', 'steam_appid': 70, 'required_age': 0, 'is_free': False, 'controller_support': 'full', 'dlc': [323130], 'detailed_description': "Named Game of the Year by over 50 publications, Valve's debut title blends action and adventure with award-winning technology to create a frighteningly realistic world where players must think to survive. Also includes an exciting multiplayer mode that allows you to play against friends and enemies around the world.", 'about_the_game': "Named Game of the Year by over 50 publications, Valve's debut title blends action and adventure with award-winning technology to create a frighteningly realistic world where players must think to survive. Also includes an exciting multiplayer mode that allows you to play against friends and enemies around the world.", 'short_description': "Named Game of the Year by over 50 publications, Valve's debut title blends action and adventure with award-winning techn